## TYTAN tutorial おすすめ8（巡回セールスマン問題）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリング（QUBO）で巡回セールスマン問題を解く](https://vigne-cla.com/21-8/)

### 問題
4つの都市A, B, C, Dを最短で訪れる経路を求める。スタートの都市は決まってない。各都市を結ぶ道の距離は分かっている。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/03/21-8_1.png" width = 40%>
</div>

### このQUBO設定を使おう

<font color="red">One-hotエンコーディング</font>

例）1～3の自然数のどれかにする（＝3個の量子ビットから1個を１にする。1になった場所を自然数に割り当てる）
```
H = (q0 + q1 + q2 - 1)**2
```

<font color="red">「2個の量子ビットが同時に1になったらペナルティを与える」</font>

例）2個の量子ビットが同時に1になったらペナルティを与える
```
H = (q0 * q1)
```

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### TSP問題のエンコーディング（≒量子ビットで表現できるように変換する）

4×4のテーブルを作り16個の量子ビットを対応させる。これはTSPの定石なので覚えるしかない。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/03/21-8_2.png" width = 50%>
</div>

### 制約条件（絶対に守ってほしい条件）

各行、各列の量子ビットは1個だけ1になってほしい。これをOne-hotと呼んだりする。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/03/21-8_3.png" width = 45%>
</div>


```
#1番目に訪れる都市は1つだけにしたい、のやつ
H = (q00 + q01 + q02 + q03 - 1)**2
H = (q04 + q05 + q06 + q07 - 1)**2
H = (q08 + q09 + q10 + q11 - 1)**2
H = (q12 + q13 + q14 + q15 - 1)**2

#都市Aに訪れる順番は1つだけにしたい、のやつ
H = (q00 + q04 + q08 + q12 - 1)**2
H = (q01 + q05 + q09 + q13 - 1)**2
H = (q02 + q06 + q10 + q14 - 1)**2
H = (q03 + q07 + q11 + q15 - 1)**2
```


### コスト（できるだけ小さくしたい≒ペナルティ）

都市間の距離に基づいた設定を行う。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/03/21-8_4.png" width = 70%>
</div>

赤矢印は1番目の都市から2番目の都市Aに行く4パターン。例えば、[q1, q4] が同時に1になることは、1番目(B)→2番目(A)を訪れることを表す。B→Aの距離3に比例したコスト（ペナルティ）を設定する（ただし制約条件よりも規模を小さくするため10分の1して0.3をかける）。

```
H = 0.3 * (q01 * q04)
※全部で48式
```

この式から、[q1, q4] が同時に１になるとエネルギーに0.3が追加される。距離に比例したコスト（ペナルティ）を設定することで、距離が長い道ほど採用しにくくなり、結果として最短経路が求まる。


## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [7]:
from tytan import *
import numpy as np

#量子ビットを用意する
q00 = symbols('q00')
q01 = symbols('q01')
q02 = symbols('q02')
q03 = symbols('q03')
q04 = symbols('q04')
q05 = symbols('q05')
q06 = symbols('q06')
q07 = symbols('q07')
q08 = symbols('q08')
q09 = symbols('q09')
q10 = symbols('q10')
q11 = symbols('q11')
q12 = symbols('q12')
q13 = symbols('q13')
q14 = symbols('q14')
q15 = symbols('q15')

#1番目に訪れる都市は1つだけにしたい、のやつ
H = 0
H += (q00 + q01 + q02 + q03 - 1)**2
H += (q04 + q05 + q06 + q07 - 1)**2
H += (q08 + q09 + q10 + q11 - 1)**2
H += (q12 + q13 + q14 + q15 - 1)**2

#都市Aに訪れる順番は1つだけにしたい、のやつ
H += (q00 + q04 + q08 + q12 - 1)**2
H += (q01 + q05 + q09 + q13 - 1)**2
H += (q02 + q06 + q10 + q14 - 1)**2
H += (q03 + q07 + q11 + q15 - 1)**2

#都市間の距離に比例したペナルティ
#1番目から2番目への移動について
H += 0.0 * (q00 * q04) #距離0なので
H += 0.3 * (q01 * q04) #距離3なので
H += 0.2 * (q02 * q04) #距離2なので
H += 0.6 * (q03 * q04) #距離6なので

H += 0.3 * (q00 * q05)
H += 0.0 * (q01 * q05)
H += 0.1 * (q02 * q05)
H += 0.2 * (q03 * q05)

H += 0.2 * (q00 * q06)
H += 0.1 * (q01 * q06)
H += 0.0 * (q02 * q06)
H += 0.3 * (q03 * q06)

H += 0.6 * (q00 * q07)
H += 0.2 * (q01 * q07)
H += 0.3 * (q02 * q07)
H += 0.0 * (q03 * q07)

#2番目から3番目への移動について
H += 0.0 * (q04 * q08)
H += 0.3 * (q05 * q08)
H += 0.2 * (q06 * q08)
H += 0.6 * (q07 * q08)

H += 0.3 * (q04 * q09)
H += 0.0 * (q05 * q09)
H += 0.1 * (q06 * q09)
H += 0.2 * (q07 * q09)

H += 0.2 * (q04 * q10)
H += 0.1 * (q05 * q10)
H += 0.0 * (q06 * q10)
H += 0.3 * (q07 * q10)

H += 0.6 * (q04 * q11)
H += 0.2 * (q05 * q11)
H += 0.3 * (q06 * q11)
H += 0.0 * (q07 * q11)

#3番目から4番目への移動について
H += 0.0 * (q08 * q12)
H += 0.3 * (q09 * q12)
H += 0.2 * (q10 * q12)
H += 0.6 * (q11 * q12)

H += 0.3 * (q08 * q13)
H += 0.0 * (q09 * q13)
H += 0.1 * (q10 * q13)
H += 0.2 * (q11 * q13)

H += 0.2 * (q08 * q14)
H += 0.1 * (q09 * q14)
H += 0.0 * (q10 * q14)
H += 0.3 * (q11 * q14)

H += 0.6 * (q08 * q15)
H += 0.2 * (q09 * q15)
H += 0.3 * (q10 * q15)
H += 0.0 * (q11 * q15)


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#上位5件
for r in result[:5]:
    print(r)
    print(np.array(list(r[0].values())).reshape(4, 4))

[{'q00': 1, 'q01': 0, 'q02': 0, 'q03': 0, 'q04': 0, 'q05': 0, 'q06': 1, 'q07': 0, 'q08': 0, 'q09': 1, 'q10': 0, 'q11': 0, 'q12': 0, 'q13': 0, 'q14': 0, 'q15': 1}, -7.499999999999999, 16]
[[1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]]
[{'q00': 0, 'q01': 0, 'q02': 0, 'q03': 1, 'q04': 0, 'q05': 1, 'q06': 0, 'q07': 0, 'q08': 0, 'q09': 0, 'q10': 1, 'q11': 0, 'q12': 1, 'q13': 0, 'q14': 0, 'q15': 0}, -7.499999999999999, 37]
[[0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]]
[{'q00': 0, 'q01': 0, 'q02': 0, 'q03': 1, 'q04': 0, 'q05': 0, 'q06': 1, 'q07': 0, 'q08': 0, 'q09': 1, 'q10': 0, 'q11': 0, 'q12': 1, 'q13': 0, 'q14': 0, 'q15': 0}, -7.3, 31]
[[0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]
[{'q00': 1, 'q01': 0, 'q02': 0, 'q03': 0, 'q04': 0, 'q05': 0, 'q06': 1, 'q07': 0, 'q08': 0, 'q09': 0, 'q10': 0, 'q11': 1, 'q12': 0, 'q13': 1, 'q14': 0, 'q15': 0}, -7.3, 16]
[[1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 0]]
[{'q00': 0, 'q01': 1, 'q02': 0, 'q03': 0, 'q04': 0, 'q05': 0, 'q06': 0, 'q07': 1, 'q08': 0, 'q09': 0

A→C→B→D、D→B→D→Aが得られた。

条件設定式が多いため、興味がある人は便利な関数やfor文を使って効率的に記述してみてほしい。チュートリアルの「温度計パズル」「お絵かきロジック」に便利な関数の使用例がある。

詳しくはドキュメント → [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)